In [1]:
%pip install transformers huggingface_hub streamlit PyPDF2 langchain langchain-community langchain_chroma sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 33.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 70.8 MB/

In [2]:
import os
import streamlit as st
from PyPDF2 import PdfReader
from langchain_chroma import Chroma
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceHub
from langchain.chains.question_answering import load_qa_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

In [4]:
def get_pdf_text(pdf_docs):
    text=""
    for pdf in pdf_docs:
        pdf_reader= PdfReader(pdf)
        for page in pdf_reader.pages:
            text+= page.extract_text()
    return  text

In [5]:
def get_text_chunks(text):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
  texts = text_splitter.split_text(text)
  return texts

In [6]:
def get_vector_store(text_chunks):
  vector_store = Chroma.from_texts(
      texts=text_chunks,
      embedding=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2'),
      persist_directory="./chroma_db"
  )

In [7]:
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
def get_conversational_chain():
  prompt_template = """
  Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
  provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
  Context:\n {context}?\n
  Question: \n{question}\n

  Answer:
  """

  prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
  model_kwargs = {"max_new_tokens":5000}
  model_id = "google/gemma-2b"
  tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
  model = AutoModelForCausalLM.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
  model.to("cuda")
  pipe = pipeline(
      "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100,device = 0
  )
  hf = HuggingFacePipeline(pipeline=pipe)
  chain = load_qa_chain(hf, chain_type="stuff", prompt=prompt)
  return chain

2024-06-19 08:41:27.845723: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-19 08:41:27.845833: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-19 08:41:27.959567: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
def user_input(user_question):
  new_db = Chroma(persist_directory="./chroma_db", embedding_function = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2'))
  docs = new_db.similarity_search(user_question,k=3)
  chain = get_conversational_chain()
  response = chain(
      {"input_documents":docs, "question": user_question}
      ,return_only_outputs=True)

  print(response)
  # st.write("Reply: ", response["output_text"])
  return response

In [9]:
# def main():
#   st.set_page_config("ChatBot with custom PDF")
#   st.header("Chat with PDF using Gemma-7b")

#   user_question = st.text_input("Ask a Question from the PDF Files")

#   if user_question:
#       user_input(user_question)

#   with st.sidebar:
#       st.title("Menu:")
#       pdf_docs = st.file_uploader("Upload your PDF Files and Click on the Submit & Process Button", accept_multiple_files=True)
#       if st.button("Submit & Process"):
#           with st.spinner("Processing..."):
#               raw_text = get_pdf_text(pdf_docs)
#               text_chunks = get_text_chunks(raw_text)
#               get_vector_store(text_chunks)
#               st.success("Done")



# if __name__ == "__main__":
#   main()

In [10]:
pdf_files = ["/kaggle/input/pdf-helper/IITM BS Degree Programme - Student Handbook - Latest.pdf"]


In [11]:
raw_text = get_pdf_text(pdf_files)
text_chunks = get_text_chunks(raw_text)
get_vector_store(text_chunks)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
res = user_input('what are the foundation course in iitm?')

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

{'output_text': '\n  Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in\n  provided context just say, "answer is not available in the context", don\'t provide the wrong answer\n\n\n  Context:\n 18\nCredit\nTransfer\nfrom\ncourses\nthat\nare\nconducted\non\nIITM\ncampus\n(**This\nwill\nbe\nin\neffect \nfrom\nJanuary\n2024**)\n19\n10.\nLearner\nLife\nCycle\n20\n11.\nDesign\nof\ncertificates\nfor\nthe\n4\nlevels\nof\nthe\nprogram\n2111.1\nMark\ntranscripts\n25\n12.\nAcademic\naspects\n25\n12.1\nAcademic\ncalendar\n25\n12.2\nCourse\nstructure\nsuitable\nfor\nonline\ndelivery\n27\n12.3\nEvaluation\nand\ngrading\nof\ncourses\nin\nevery\nterm\n27\n12.4\nPass\ncriteria\nfor\neach\ncourse\n28\n12.5\nRepeating\na\ncourse\n29\n12.5.1\nRegistering\nfor\nCourses:\nTerm\n2\nOnwards\n30\n12.5.2\nRepeating\na\nCourse\n-\nWA,\nWQ,\nU\nor\nI\ngrade\n30\n12.5.3\nRepeating\na\nCourse\nfor\nImprovement\n30\n12.5.4\nMake\nUp\n

In [16]:
print(res['output_text'])


  Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
  provided context just say, "answer is not available in the context", don't provide the wrong answer


  Context:
 18
Credit
Transfer
from
courses
that
are
conducted
on
IITM
campus
(**This
will
be
in
effect 
from
January
2024**)
19
10.
Learner
Life
Cycle
20
11.
Design
of
certificates
for
the
4
levels
of
the
program
2111.1
Mark
transcripts
25
12.
Academic
aspects
25
12.1
Academic
calendar
25
12.2
Course
structure
suitable
for
online
delivery
27
12.3
Evaluation
and
grading
of
courses
in
every
term
27
12.4
Pass
criteria
for
each
course
28
12.5
Repeating
a
course
29
12.5.1
Registering
for
Courses:
Term
2
Onwards
30
12.5.2
Repeating
a
Course
-
WA,
WQ,
U
or
I
grade
30
12.5.3
Repeating
a
Course
for
Improvement
30
12.5.4
Make
Up
Exams
30
12.6
Dropping
a
course
(This
is
not
applicable
for
the
students
who
are
continuing
after 
qualifier
in
the
same
term)
32
13

In [14]:
!streamlit run app.py

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py
